In [1]:
# workspace = "dev"
# apiVersion = "5.2"
# moduleName = "Ed-Fi10"
# schoolYear = None
# districtId = None

StatementMeta(spark3p3sm, 208, 2, Finished, Available)

In [2]:
%run /edfi_py

StatementMeta(, 208, -1, Finished, Available)

2023-07-24 14:01:26,102 - OEA - INFO - Now using workspace: dev
2023-07-24 14:01:26,104 - OEA - INFO - OEA initialized.
2023-07-24 14:01:26,988 - OEA - INFO - Now using workspace: dev
2023-07-24 14:01:27,399 - EdFiClient - INFO - minChangeVersion=None and maxChangeVersion=None


In [3]:
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"

# parameterized = True

StatementMeta(spark3p3sm, 208, 9, Finished, Available)

In [8]:
if parameterized == True:
    edfiEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi/parameterizedRunLogs/currentRun'
    tpdmEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm/parameterizedRunLogs/currentRun'

    _, edfiEntities = edfi.listSpecifiedEntities(edfiEntitiesPath)
    _, tpdmEntities = edfi.listSpecifiedEntities(tpdmEntitiesPath)
else:
    edfiEntities = "All"
    tpdmEntities = "All"   

StatementMeta(spark3p3sm, 208, 14, Finished, Available)

In [14]:
print(edfiEntities + tpdmEntities)

StatementMeta(spark3p3sm, 208, 20, Finished, Available)

['absenceEventCategoryDescriptors', 'academicHonorCategoryDescriptors']


In [12]:
def ingest_edfi_dataset(tables_source, 
                        items = 'All'):
    full_source_path = f"stage1/Transactional/{tables_source}"
    if items == 'All':
        items = oea.get_folders(full_source_path)          
    options = {'header':False}
    for item in items: 
        table_path = full_source_path +'/'+ item
        try:
            entity_path = f"{tables_source}/{item}"
            print(entity_path)
            if item == 'metadata.csv' or item == 'parameterizedRunLogs':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                oea.ingest(entity_path, 'id', options)
        except AnalysisException as e:
            # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
            pass

StatementMeta(spark3p3sm, 208, 18, Finished, Available)

In [13]:
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi',
                    edfiEntities)
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm',
                    tpdmEntities)

StatementMeta(spark3p3sm, 208, 19, Finished, Available)

Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors
2023-07-24 14:05:17,121 - EdFiClient - INFO - Ingesting from: stage1/Transactional/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors, batch type of: delta, source data format of: json
source_path is: abfss://oea@stoeahisddev.dfs.core.windows.net/dev/stage1/Transactional/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors/delta_batch_data
2023-07-24 14:05:34,664 - py4j.java_gateway - INFO - Callback Server Starting
2023-07-24 14:05:34,664 - py4j.java_gateway - INFO - Socket listening on ('127.0.0.1', 35721)
2023-07-24 14:05:36,523 - EdFiClient - INFO - Number of new inbound rows processed: 0
Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryDescriptors
2023-07-24 14:05:36,699 - EdFiClient - INFO - Ingesting from: stage1/Transactional/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryDescriptors, batch type of: delta, source